In [1]:
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Input, Dropout, Dense, Flatten, Reshape, Conv1D, MaxPooling1D, GRU, TimeDistributed
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import numpy as np

from code.util import *
from code.modelhelper import *

In [2]:
VLEN = 128
BATCH_SIZE = 512
EPOCHS = 100

In [3]:
x = np.load("data/x_char_onehot_67000.npz")["arr_0"]
y = np.load("data/y_onehot_67000.npy")

In [4]:
x_train, x_test, y_train, y_test = train_test_split(
    x, y, test_size=0.3, random_state=42
)

In [5]:
model_input = Input(shape=(x_train.shape[1:]))
H = Reshape((128, x_train.shape[1]//VLEN, 1))(model_input)
H = TimeDistributed(Conv1D(filters=20, kernel_size=(2), padding='same', activation='tanh'))(H)
H = TimeDistributed(MaxPooling1D(2))(H)
H = TimeDistributed(Conv1D(filters=20, kernel_size=(2), padding='same', activation='tanh'))(H)
H = TimeDistributed(MaxPooling1D(2))(H)
H = TimeDistributed(Conv1D(filters=20, kernel_size=(2), padding='same', activation='tanh'))(H)
H = TimeDistributed(MaxPooling1D(2))(H)
H = TimeDistributed(Flatten())(H)
H = Dropout(0.5)(H)
H = GRU(256, return_sequences=True)(H)
H = GRU(256, return_sequences=False, go_backwards=True)(H)
H = Dropout(rate=0.5)(H)
H = Dense(64, activation='tanh')(H)
model_output = Dense(2, activation='softmax')(H)

model = Model(model_input, model_output)
model.summary()

model.compile(
    loss="categorical_crossentropy",
    optimizer="adam",
    metrics=["accuracy"]
)

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 18816)]           0         
_________________________________________________________________
reshape (Reshape)            (None, 128, 147, 1)       0         
_________________________________________________________________
time_distributed (TimeDistri (None, 128, 147, 20)      60        
_________________________________________________________________
time_distributed_1 (TimeDist (None, 128, 73, 20)       0         
_________________________________________________________________
time_distributed_2 (TimeDist (None, 128, 73, 20)       820       
_________________________________________________________________
time_distributed_3 (TimeDist (None, 128, 36, 20)       0         
_________________________________________________________________
time_distributed_4 (TimeDist (None, 128, 36, 20)       820   

In [6]:
es = EarlyStopping(
    monitor="val_loss",
    patience=5,
    restore_best_weights=True
)

history = model.fit(
    x_train, y_train,
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    verbose=1,
    validation_data=(x_test, y_test),
    callbacks=[es]
)

score = model.evaluate(x_test, y_test, verbose=0)

W0821 15:43:06.159702 4529051072 deprecation.py:323] From /Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 46900 samples, validate on 20100 samples
Epoch 1/100
 3072/46900 [>.............................] - ETA: 8:31 - loss: 0.6470 - accuracy: 0.6423

KeyboardInterrupt: 

In [ ]:
print('Test loss:', score[0])
print('Test accuracy:', score[1])

y_pred_model = model.predict(x_test)
y_pred = to_bin(y_pred_model)
print("Test recall: {}".format(recall(y_test, y_pred)))

y_test0 = to_1D(y_test)
y_pred0 = to_1D(y_pred)
print(confusion_matrix(y_test0, y_pred0))

In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(history.history["accuracy"], label="Train")
plt.plot(history.history["val_accuracy"], label="Test")
plt.legend()
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.show()

In [ ]:
model.save("models/cnn_char.h5")